In [21]:
import pandas as pd

# 使用相对路径读取目录下breachlist.pkl文件
df = pd.read_pickle('breachlist.pkl')

url = df['href'][0]

from pyquery import PyQuery as pq


import pandas as pd

# 使用相对路径读取目录下breachlist.pkl文件
df = pd.read_pickle('breachlist.pkl')

url = df['href'][3]



def get_details(url):
    try:
        # 使用 PyQuery 解析 url 中的 table 标签
        doc = pq(url)
        table = doc('table')
        # 将 table 转换为 Pandas DataFrame
        tdf = pd.read_html(str(table))[0]
        # 获取 table 标签下的所有子级 a 标签的 href 属性值
        hrefs = table.find('a')
        clist = []
        dlist = []
        for href in hrefs:
            # 判断属性值是否包含 "case_details" 或 "defendant_details" 字符
            if 'case_details' in href.attrib['href']:
                clist.append('https://resources.hse.gov.uk/convictions-history/case/' + href.attrib['href'])
            elif 'defendant_details' in href.attrib['href']:
                dlist.append('https://resources.hse.gov.uk/convictions-history/defendant/' + href.attrib['href'])
            else:
                pass
        # 返回 url 为 key，tdf 为 value 的字典，以及 clist 和 dlist
        return {url: tdf}, clist, dlist
    except:
        # 解析失败或异常时，抛出异常返回 None
        raise Exception('Failed to parse URL')


# 调用 get_details 方法，将返回值分别赋值给 resdic, clist, dlist
resdic, clist, dlist = get_details(url)


# 如果 resdic 不等于 None
if resdic is not None:
    # 如果 clist 元素大于0
    if len(clist) > 0:
        # 将 clist 中的每个元素都传入 get_details 方法中，只保留第一个返回值，过滤掉为 None 的情况，结果保存为变量 clist_res
        clist_res = [get_details(url)[0] for url in clist if get_details(url)[0] is not None][0]
    # 如果 clist 元素等于0
    else:
        clist_res = None
    # 如果 dlist 元素大于0
    if len(dlist) > 0:
        # 将 dlist 中的每个元素都传入 get_details 方法中，只保留第一个返回值，过滤掉为 None 的情况，结果保存为变量 dlist_res
        dlist_res = [get_details(url)[0] for url in dlist if get_details(url)[0] is not None][0]
    # 如果 dlist 元素等于0
    else:
        dlist_res = None

# Combine all dictionaries in resdic, clist_res, and dlist_res into one dictionary
combined_dict = {}
for d in [resdic, clist_res, dlist_res]:
    if d is not None:
        combined_dict.update(d)
        
# The combined dictionary is now stored in the variable combined_dict.
import pickle

# 将 combined_dict 通过pickle 本地序列化
with open('combined_dict.pkl', 'wb') as f:
    pickle.dump(combined_dict, f)


In [20]:
list(combined_dict.values())[0]

,Case No. 4687342 Details for breach 01,Case No. 4687342 Details for breach 01.1,Case No. 4687342 Details for breach 01.2,Unnamed: 3
0,Defendant,Holt Estates England Limited,Holt Estates England Limited,Holt Estates England Limited
1,Court Name,NaN,Court Level,Magistrates Court
2,Act,"Health and Safety At Work Act 1974, Section 3,...","Health and Safety At Work Act 1974, Section 3,...","Health and Safety At Work Act 1974, Section 3,..."
3,Regulation,NaN,NaN,NaN
4,Date of Hearing,04/04/2022,NaN,NaN
5,Result,Fine,NaN,NaN
6,Fine,"£40,000.00",NaN,NaN
7,Case Details,Case Details,Case Details,Case Details
8,Location of Offence,Location of Offence,Location of Offence,Location of Offence
9,Address,Orchards2 Ash LaneChardTA20 2LZEngland,Region,Wales & South West
